In [1]:
import json, pandas as pd, pprint 

cdir = "/media/jsaxon/brobdingnag/data/s3/res/"
jdir = "/media/jsaxon/brobdingnag/data/c4_redux/"

def write_alternate_votes(s, vote_mod):

    votes = pd.read_csv("voting/votes/{}_{}.csv".format(s, vote_mod), index_col = "rn")
    years = set(votes.columns.str.replace("R", "").str.replace("D", "").astype(int) + 2000)

    json_output = open(jdir + "/{}_{}_redux.json".format(s, vote_mod), "w")
    with open(jdir + "/{}_redux.json".format(s)) as fi:
        for li, line in enumerate(fi):

            if not li % 1000: print(li//1000, end = " ", flush = True)
            j = json.loads(line)

            fcsv = cdir + j["UID"] + "/final.csv"
            df = pd.read_csv(fcsv, names = ["rn", "cd"], index_col = "rn")
            df = df.join(votes).groupby("cd").sum()

            DemSeats, RepFrac = {}, []
            for y in years:

                y = "{:02d}".format(y - 2000)

                DemSeats["20" + y] = (df["D" + y] > df["R" + y]).sum()

                df["T" + y] = df["D" + y] + df["R" + y]
                RepFrac.extend(list((df["R" + y] / df["T" + y]).round(3)))


            jout = {"UID"                 : j["UID"],
                    "Score"               : round(j["Score"], 3),
                    "PopulationDeviation" : round(j["PopulationDeviation"], 3),
                    "DemSeats" : DemSeats, "RepFrac": RepFrac}

            json_output.write(str(jout).replace("'", '"') + "\n")


    json_output.close()

In [2]:
# write_alternate_votes("tx", "senate")
write_alternate_votes("tx", "presidential")
write_alternate_votes("va", "presidential")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 